# Extraer lista de identificadores de twitch de las categorias más vistas (Haciendo scrapping)

In [2]:
# Para la extracción de los identificadores de las categorias y los streamers
import pandas as pd
import sys
sys.path.append("../")
from src import soporte_extraccion as se

# Para la extracción de datos ampliados
import time
from time import sleep

# Para la extracción de datos de las categorias asincrono
from src import extraccion_async_info_categorias as eic

# Para la extracción de datos de los streamers asincrono
from src import extraccion_async_info_streamers as eis

# Para el almacenamiento en base de datos
from src import soporte_bbdd_mongo as sbm

# Otros
%reload_ext autoreload
%autoreload 2

In [ ]:
df_categorias = se.extraer_lista_categorias(silent_mode=True)

df_categorias = se.cambiar_tipos_columnas(df_categorias, df_categorias.columns.to_list())

df_identificadores_categorias = se.obtener_identificadores(df_categorias)

# Almacenamiento en base de datos
client, db = sbm.conectar_mongo()
sbm.insertar_en_coleccion(db, "seven_viewers", df_categorias)
sbm.insertar_en_coleccion(db, "id_categorias", df_identificadores_categorias)
client.close()

# Almacenamiento en csv y pickle de los datos
# df_identificadores_categorias.to_pickle("../datos/raw/categorias/identificadores/categories_id.pkl")
# df_identificadores_categorias.to_csv("../datos/raw/categorias/identificadores/categories_id.csv")
# df_categorias.to_pickle("../datos/raw/categorias/ultimos_7_dias/viewers_last_seven_days_all_categories.pkl")
# df_categorias.to_csv("../datos/raw/categorias/ultimos_7_dias/viewers_last_seven_days_all_categories.csv")

In [ ]:
# Ampliación de extracción de los datos
# lista_elementos_clickar = ["rating", "average-channels", "time-watched", "live", "peak-viewers", "peak-channels", "to-stream"]
lista_elementos_clickar = ["rating", "average-channels", "time-watched", "peak-viewers", "peak-channels", "to-stream"]
client, db = sbm.conectar_mongo()
for endpoint in lista_elementos_clickar:
    print(f"\nIniciando {endpoint.capitalize()}\n")
    inicio = time.time()
    df_categorias = se.extraer_lista_categorias_ampliado(endpoint, silent_mode=True)
    tiempo = round(time.time() - inicio, 1)
    print(f"\nHa tardado {tiempo} segundos\n")
    #display(df_categorias.head())
    df_categorias.to_pickle(f"../datos/raw/categorias/ultimos_7_dias/{endpoint.replace("-", "_")}_last_seven_days_all_categories.pkl")
    df_categorias.to_csv(f"../datos/raw/categorias/ultimos_7_dias/{endpoint.replace("-", "_")}_last_seven_days_all_categories.csv")
    print(f"\nDatos almacenados en: datos/raw/categorias/ultimos_7_dias/{endpoint.replace("-", "_")}_last_seven_days_all_categories.csv\n")
    sbm.insertar_en_coleccion(db, f"seven_{endpoint.replace("-", "_")}", df_categorias)
    sleep(5)
client.close()

# Extracción de info sobre todas las categorias de la lista (Async)

In [4]:
# # Este es el que funciona
# import pandas as pd
# import sys
# sys.path.append("../")
# from src import extraccion_async_info_categorias as pa

# %reload_ext autoreload
# %autoreload 2

In [ ]:
# # Margen de error con 8 nucleos y 16 retires = 
# # Margen de error con 4 nucleos y 16 retries = 901/930 (se quedó bloqueado en la última y no hizo el backup)
# # Margen de error con 4 hilos y 4 retries = (Le he metido un timeout de 2 minutos)
eic.main()

In [6]:
# ruta_backup = "../datos/raw/categorias/backup/backup"
# ruta_carpeta = "../datos/output/categorias/all"
# se.consolidar_backup(ruta_carpeta, ruta_backup)

# Extracción de info sobre las categorias de la lista (No async)

In [ ]:
# data_pickle = pd.read_pickle(f"../datos/raw/categorias/identificadores/categories_id.pkl")
# df_identificadores_categorias = pd.DataFrame(data_pickle)
# df_final = se.extraer_info_categoria_a_categoria(df_identificadores_categorias, silent_mode=True)
# df_final.to_csv("../datos/raw/categorias/backup/prueba_backup_categorias.csv")
# df_final.to_pickle("../datos/raw/categorias/backup/prueba_backup_categorias.pkl")

# Streamers

### Extracción de la lista de streamers (No async)

In [2]:
df_streamers = se.extraer_lista_streamers(True)
display(df_streamers.head(-10))
client, db = sbm.conectar_mongo(local=False)
sbm.insertar_en_coleccion(db, "streamers", df_streamers)
client.close()

100%|██████████| 50/50 [47:13<00:00, 56.66s/it]


,Nombre,Enlace


✅ Datos insertados o actualizados en la colección 'streamers' correctamente.


### Streamers uno a uno

In [4]:
data_pickle = pd.read_pickle(f"../datos/raw/streamers/streamers_ranking.pkl")
df_streamers = pd.DataFrame(data_pickle)
df_streamers = df_streamers[600:1400] # Hasta 1400

In [3]:
df_datos_streamers = se.extraer_info_stremaer_a_streamer(df_streamers, silent_mode=True)
# df_datos_streamers.to_pickle(f"../datos/raw/streamers/backup/backup_datos_streamers.pkl")
# df_datos_streamers.to_csv(f"../datos/raw/streamers/backup/backup_datos_streamers.csv")

100%|██████████| 70/70 [44:37<00:00, 38.25s/it]


# Streamers async

In [ ]:
# eis.main()

In [ ]:
# ruta_backup = "../datos/raw/streamers/backup/backup"
# ruta_carpeta = "../datos/output/streamers/all"
# se.consolidar_backup(ruta_carpeta, ruta_backup)

In [3]:
import glob
import csv
import os
from tqdm import tqdm

# Ruta de la carpeta que contiene los archivos CSV
carpeta_csv = '../datos/output/streamers/all'  # <-- Cambia esta ruta según tu entorno

# Genera la lista de todos los archivos CSV en la carpeta
patron = os.path.join(carpeta_csv, '*.csv')
lista_csv = glob.glob(patron)


if not lista_csv:
    print("No se encontraron archivos CSV en la carpeta especificada.")
else:
    client, db = sbm.conectar_mongo(False)
    # Procesa cada archivo CSV encontrado
    for archivo in tqdm(lista_csv):
        # nombre = os.path.splitext(os.path.basename(archivo))[0].strip().lower()
        # nombre = nombre.split("_")[2]
        # df.insert(0, "Nombre", nombre)
        df = pd.read_csv(archivo, index_col=0)

        if "id_streamer" in df.columns:
            df.drop(columns=["id_streamer"], inplace=True)

        if f"% Gain" in df.columns:
            df.rename(columns={f"% Gain": "PercentageGainHours"}, inplace=True)
            df["PercentageGainHours"] = df["PercentageGainHours"].str.replace("%", "")

        col_aux = df.copy()

        if f"FollowesGain" in df.columns:
            df.rename(columns={f"FollowesGain": "FollowersGain"}, inplace=True)
            df["PercentageGainFollowers"] = df["FollowersGain"]
            df["FollowersGain"] = col_aux["PercentageGainFollowers"]
            df["PercentageGainFollowers"] = df["PercentageGainFollowers"].str.replace("%", "").str.replace("+", "").str.replace(r"\-", "0", regex=True).str.replace(",", "").astype(float)
            df["FollowersGain"] = df["FollowersGain"].astype(str).str.replace("+", "").str.replace(",", "").str.replace(r"\-", "0", regex=True).astype(int)
            df["Followers"] = df["Followers"].astype(str).str.replace(",", "").astype(int)


        df = se.asignar_ids_streamers(df)
        
        if df is not None:
            sbm.insertar_en_coleccion(db, "historico_streamers", df, True, None)
        else:
            pass
    
    client.close()
    

  2%|▏         | 44/2043 [00:17<15:00,  2.22it/s]

Error: El DataFrame no contiene la columna 'Nombre'. No se pueden asignar IDs.


 92%|█████████▏| 1882/2043 [12:22<01:09,  2.31it/s]

Error: El DataFrame no contiene la columna 'Nombre'. No se pueden asignar IDs.


100%|██████████| 2043/2043 [13:25<00:00,  2.54it/s]


In [4]:
import glob
import csv
import os
from tqdm import tqdm

# Ruta de la carpeta que contiene los archivos CSV
carpeta_csv = '../datos/output/streamers/cats_streams/'  # <-- Cambia esta ruta según tu entorno

# Genera la lista de todos los archivos CSV en la carpeta
patron = os.path.join(carpeta_csv, '*.csv')
lista_csv = glob.glob(patron)


if not lista_csv:
    print("No se encontraron archivos CSV en la carpeta especificada.")
else:
    client, db = sbm.conectar_mongo(False)
    # Procesa cada archivo CSV encontrado
    for archivo in tqdm(lista_csv):
        # nombre = os.path.splitext(os.path.basename(archivo))[0].strip().lower()
        # nombre = nombre.split("_")[2]
        # df.insert(0, "Nombre", nombre)
        df = pd.read_csv(archivo, index_col=0)

        if "id_streamer" in df.columns:
            df.drop(columns=["id_streamer"], inplace=True)

        if f"% Gain" in df.columns:
            df.rename(columns={f"% Gain": "PercentageGainHours"}, inplace=True)

        if f"FollowesGain" in df.columns:
            df.rename(columns={f"FollowesGain": "FollowersGain"}, inplace=True)

        df = se.asignar_ids_streamers(df)

        # # Solo subo la primera categoria
        # df_categorias_group = df.groupby("id_streamer").first().reset_index()
        
        if df is not None:
            sbm.insertar_en_coleccion(db, "categorias_streameadas", df, True, None)
        else:
            pass
    
    client.close()

100%|██████████| 2039/2039 [06:37<00:00,  5.13it/s]


In [5]:
import glob
import csv
import os
from tqdm import tqdm

# Ruta de la carpeta que contiene los archivos CSV
archivo = '../datos/raw/streamers/streamers_ranking.pkl'  # <-- Cambia esta ruta según tu entorno

client, db = sbm.conectar_mongo()
# nombre = os.path.splitext(os.path.basename(archivo))[0].strip().lower()
# nombre = nombre.split("_")[2]
# df.insert(0, "Nombre", nombre)
data_pickle = pd.read_pickle(archivo)
df = pd.DataFrame(data_pickle)

if "id_streamer" in df.columns:
    df.drop(columns=["id_streamer"], inplace=True)

if f"% Gain" in df.columns:
    df.rename(columns={f"% Gain": "PercentageGainHours"}, inplace=True)

if f"FollowesGain" in df.columns:
    df.rename(columns={f"FollowesGain": "FollowersGain"}, inplace=True)

df = se.asignar_ids_streamers(df)

sbm.insertar_en_coleccion(db, "streamers", df, True, "id_streamer")

client.close()

In [ ]:
# Extracción de datos adicionales desde la API
import requests

url = "https://api.twitchtracker.com/v1/endpoint"
response = requests.get(url)
datos = response.json()
client, db = sbm.conectar_mongo(local=False)
sbm.insertar_en_coleccion(db, "last_30_days", datos, clave_unica="nombre")
client.close()

In [1]:
import sys
sys.path.append("../")
from src import extraccion_categorias_horas as ech
import pandas as pd

df_streamers = pd.read_csv("../datos/raw/streamers/ids_streamers.csv").head(5)
df_datos, df_categorias = ech.extraer_info_streamer_joblib(df_streamers, concatenar=True)


ProtocolError: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))

In [ ]:
df_datos

In [ ]:
df_categorias